Q1. What is Gradient Boosting Regression?
--
---
Gradient Boosting Regression is a machine learning technique that belongs to the class of ensemble methods, specifically falling under the category of boosting algorithms. Gradient Boosting is used for both classification and regression tasks, but when it's applied to regression problems, it's referred to as Gradient Boosting Regression.

The general idea behind Gradient Boosting Regression is to sequentially build a series of weak learners, usually decision trees, and combine them to create a strong predictive model. Unlike traditional decision tree models, which are built in parallel, boosting algorithms build trees sequentially, with each tree aiming to correct the errors made by the previous ones.

Q2. Implement a simple gradient boosting algorithm from scratch using Python and NumPy. Use a simple regression problem as an example and train the model on a small dataset. Evaluate the model's  performance using metrics such as mean squared error and R-squared.
--

In [3]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor


np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)


X_train, X_test, y_train, y_test = X[:80], X[80:], y[:80], y[80:]

class GradientBoosting:
    def __init__(self, S=5, learning_rate=1, max_depth = 1, min_samples_split = 2, regression=True):
        self.S = S
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.regression=regression
        
        
        tree_params = {'max_depth': self.max_depth, 'min_samples_split': self.min_samples_split}
        self.models = [DecisionTreeRegressor(**tree_params) for _ in range(S)]        
   
    def grad(self, y, h):
        return y - h

    def fit(self, X, y):  
        # Fit the first mode
        self.models[0].fit(X, y)
        
        for i in range(self.S - 1):

            yhat = self.predict(X, self.models[:i+1], with_argmax=False)
            
            # Get the gradient
            gradient = self.grad(y, yhat)
            
            # Fit the next model with gradient
            self.models[i+1].fit(X, gradient)
    
    def predict(self, X, models=None, with_argmax=True):
        if models is None:
            models = self.models
        h0 = models[0].predict(X)  # first use the initial model
        boosting = sum(self.learning_rate * model.predict(X) for model in models[1:])
        yhat = h0 + boosting
        if not self.regression:
            # Turn into probability distribution (Softmax)
            yhat = np.exp(yhat) / np.sum(np.exp(yhat), axis=1, keepdims=True)
            if with_argmax:
                yhat = np.argmax(yhat, axis=1)
        return yhat

# Initialize model
model = GradientBoosting(S=200, learning_rate=0.1, max_depth = 3, min_samples_split = 2)


model.fit(X_train, y_train)


y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse:.2f}')
print(f'R2 Score: {r2:.2f}')


MSE: 0.00
R2 Score: 0.92


Q4. What is a weak learner in Gradient Boosting?
--
---
In the context of gradient boosting, a weak learner is a simple machine learning model that is only slightly better than random guessing. Weak learners are typically used in ensemble learning algorithms, where they are combined to create a more accurate and powerful prediction model.

The use of weak learners in gradient boosting is based on the idea that it is easier to train a model to make small improvements than to train a model to make large improvements. By combining many weak learners, each of which makes a small improvement, it is possible to create a model that is much more accurate than any of the individual weak learners.

Q5. What is the intuition behind the Gradient Boosting algorithm?
--
---
The intuition behind the Gradient Boosting algorithm is to combine several weak learners to create a strong learner that can make accurate predictions. Here's a step-by-step breakdown of the intuition:

1. **Start with a Weak Learner**: The algorithm starts with a weak learner, typically a decision tree, which is just slightly better than random guessing.

2. **Calculate Errors**: The algorithm calculates the errors or residuals, which are the differences between the actual and predicted values.

3. **Train Another Weak Learner**: The algorithm then trains another weak learner to predict these residuals instead of the actual values. This new weak learner is therefore focused on correcting the mistakes of the first weak learner.

4. **Update Predictions**: The predictions of the model are updated using the predictions of this new weak learner. This means that the new weak learner's predictions are added to the previous weak learner's predictions to get the final prediction.

5. **Repeat**: Steps 2-4 are repeated for a specified number of iterations. With each iteration, a new weak learner is trained to correct the residuals of the previous weak learner.

6. **Final Model**: The final model is a weighted sum of all the weak learners. Each weak learner in the ensemble focuses on the errors of the previous one, thereby improving the predictions of the model with each iteration.

Q6. How does Gradient Boosting algorithm build an ensemble of weak learners?
--
---
The Gradient Boosting algorithm builds an ensemble of weak learners in a stage-wise fashion. Here's how it works:

1. **Initialize the Model**: The algorithm starts by predicting the average of the target variable.
2. **Compute Residuals**: The residuals (differences) between the predicted and actual values are computed.
3. **Train a Weak Learner**: A weak learner (typically a decision tree) is trained to predict the residuals instead of the actual values.
4. **Update Predictions**: The predictions of the model are updated using the predictions of the weak learner.
5. **Repeat**: Steps 2-4 are repeated for a specified number of iterations.

The final model is a weighted sum of the weak learners. Each weak learner in the ensemble focuses on the errors of the previous one, thereby improving the predictions of the model with each iteration.

In contrast to AdaBoost, the weights of the training instances are not tweaked, instead, each predictor is trained using the residual errors of the predecessor as labels. There is a technique called the Gradient Boosted Trees whose base learner is CART (Classification and Regression Trees).

There is an important parameter used in this technique known as Shrinkage. Shrinkage refers to the fact that the prediction of each tree in the ensemble is shrunk after it is multiplied by the learning rate (eta) which ranges between 0 to 1. There is a trade-off between eta and the number of estimators, decreasing learning rate needs to be compensated with increasing estimators in order to reach certain model performance.

Q7. What are the steps involved in constructing the mathematical intuition of Gradient Boosting algorithm?
--
----

The mathematical intuition behind the Gradient Boosting algorithm can be understood through the following steps:

1. **Initialize the Model**: The algorithm starts by predicting the average of the target variable, which can be represented mathematically as $$F_0(x) = \frac{1}{N}\sum_{i=1}^{N}y_i$$ where $$F_0(x)$$ is the initial prediction, $$N$$ is the number of instances, and $$y_i$$ is the target value for the $$i^{th}$$ instance.

2. **Compute Residuals**: The residuals (differences) between the predicted and actual values are computed. The residual for the $$i^{th}$$ instance can be calculated as $$r_{i1} = y_i - F_0(x_i)$$ where $$r_{i1}$$ is the residual, $$y_i$$ is the actual value, and $$F_0(x_i)$$ is the predicted value.

3. **Train a Weak Learner**: A weak learner (typically a decision tree) is trained to predict these residuals instead of the actual values. This can be represented as $$h_1(x_i) = r_{i1}$$ where $$h_1(x_i)$$ is the prediction of the first weak learner.

4. **Update Predictions**: The predictions of the model are updated using the predictions of the weak learner. The updated prediction can be calculated as $$F_1(x_i) = F_0(x_i) + \nu h_1(x_i)$$ where $$F_1(x_i)$$ is the updated prediction, $$F_0(x_i)$$ is the old prediction, $$\nu$$ is the learning rate, and $$h_1(x_i)$$ is the prediction of the weak learner.

5. **Repeat**: Steps 2-4 are repeated for a specified number of iterations. With each iteration, a new weak learner is trained to correct the residuals of the previous weak learner, and the predictions are updated accordingly.

6. **Final Model**: The final model is a weighted sum of all the weak learners. It can be represented as $$F_M(x) = F_0(x) + \sum_{m=1}^{M}\nu h_m(x)$$ where $$F_M(x)$$ is the final prediction, $$F_0(x)$$ is the initial prediction, $$\nu$$ is the learning rate, $$h_m(x)$$ is the prediction of the $$m^{th}$$ weak learner, and $$M$$ is the total number of weak learners.